<a href="https://colab.research.google.com/github/anqi-guo/duplicated_complaints_identification/blob/main/2_preprocess_unlabeled_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

raw_data_path = '/content/drive/My Drive/work/重复线索识别/data/raw_data'
stopwords_path = '/content/drive/My Drive/work/重复线索识别/data/stopwords'
unlabeled_data_path = '/content/drive/My Drive/work/重复线索识别/data/unlabeled_data'

import pandas as pd
orig_df = pd.read_csv(f'{raw_data_path}/raw_data.csv')
orig_df.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(80101, 16)

In [ ]:
df = orig_df[~orig_df['XSNR'].str.contains('前期工单|原工单|已有工单|前单号')][['BH','XSBH','WRLX','DSMC','XZQH','XSNR']]
df.shape

(71849, 6)

In [ ]:
!pip install hanlp[full]
!pip install sentence_transformers
!pip install folium==0.2.1
!pip install imgaug==0.2.5

     |████████████████████████████████| 69 kB 3.5 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=dea0c5990716740bf0090e2d4415b3f91e2be9d20e795b1e637f3981918cb6f7
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 562 kB 5.4 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=4a309a1bdb09bb7dbce319e6348c205a347d53a182a8d5530107d8ea2fa01305
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
import hanlp
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
ner = hanlp.load(hanlp.pretrained.ner.MSRA_NER_ELECTRA_SMALL_ZH)

import regex as re
import difflib
from itertools import product
import json
import jieba

def extract_location(sentence):
    nerred_sent = ner(tok(sentence))
    # 提取命名实体是ORGANIZATION或者LOCATION，且结尾不是市镇区县的词
    locations = [l[0] for l in nerred_sent if (l[1]=='ORGANIZATION' or l[1]=='LOCATION') and len(re.findall('[市镇区县]$', l[0]))==0]

    return locations

def extract_problem(text):
  # 删掉标点符号
  text = re.sub(r'\p{P}+', '', text) 
  # 删掉空格、换行符、信息保密、信息不保密、距离、时间、数字、投诉人信息
  text = re.sub(r'(\s|\n|信息[不]?保密|\d+[m米年月日]?|(.{1}先生|.{1}女士|市民)(来电)?反映)', '', text) 

  if '问题描述' in text and '诉求目的' in text:
    text = re.search(r'(?<=问题描述).*(?=诉求目的)', text).group()

  # 删掉停用词
  with open(f'{stopwords_path}/cn_stopwords.txt') as f:
    stopwords = f.read().splitlines() 

  seg_list = jieba.cut(text)

  result = ''.join([s for s in seg_list if s not in stopwords])
  return result

def extract_loupan(text):
  loupan_list = []
  address_list = []

  with open(f'{raw_data_path}/新楼盘.json') as json_file:
    loupan = json.load(json_file)

  with open(f'{raw_data_path}/旧楼盘.json') as json_file:
    old_loupan = json.load(json_file)

  loupan.update(old_loupan)

  for lp, addr in loupan.items():
    if lp in text:
      loupan_list.append(lp)
      address_list.append(addr)

  return loupan_list, address_list

def get_similar_ratio(list_1, list_2):
    unique_combinations = list(product(list_1, list_2))
    similar_ratios = [difflib.SequenceMatcher(None, l[0], l[1]).ratio() for l in unique_combinations]
    count = sum(map(lambda x : x>.8, similar_ratios))
    return count

def find_similar_pairs(row1, row2):
    is_similar = False
    # 如果线索内容完全相同，则pass
    if row1['XSNR'] == row2['XSNR']:
        pass
    # 如果线索内容包含相同楼盘，或者相同的location个数大于2，或者相似的location个数大于3，或者location与楼盘地址相似个数大于2，则认为可能是重复线索
    elif len(set(row1['loupan']).intersection(set(row2['loupan']))) > 0 \
    or len(set(row1['location']).intersection(set(row2['location']))) > 2 \
    or get_similar_ratio(row1['location'], row2['location']) > 3 \
    or get_similar_ratio(row1['location'], row2['loupan_address']) > 2:
        is_similar = True
    
    if is_similar == True:
        return [row1['problem'], row2['problem']]
    else:
        return None

In [ ]:
from collections import OrderedDict
xzqh_cnt = {}

for i in df['XZQH'].unique():
  xzqh_cnt[i] = len(df[df['XZQH']==i])

xzqh_cnt = OrderedDict(sorted(xzqh_cnt.items(), key = lambda x: x[1], reverse = False))
xzqh_cnt

OrderedDict([(nan, 0),
             ('宝应湖农场', 5),
             ('清江浦区淮安现代商务集聚区管委会', 18),
             ('金港路街道', 20),
             ('清江浦区清河经济开发区管委会', 23),
             ('金湖经济开发区', 29),
             ('岔庙镇', 33),
             ('老子山镇', 34),
             ('大东镇', 37),
             ('高铁商务区街道', 39),
             ('南集镇', 40),
             ('唐集镇', 43),
             ('金南镇', 48),
             ('前锋镇', 49),
             ('石湖镇', 49),
             ('清江浦区工业园', 53),
             ('黄营镇', 53),
             ('陈师街道', 53),
             ('吕良镇', 55),
             ('成集镇', 56),
             ('东胡集镇', 56),
             ('戴楼街道', 58),
             ('淮洪路街道', 62),
             ('流均镇', 64),
             ('洪泽经济开发区', 67),
             ('三河镇', 68),
             ('宁连路街道', 69),
             ('金北街道', 73),
             ('梁岔镇', 75),
             ('桂五镇', 77),
             ('张码街道', 77),
             ('岔河镇', 78),
             ('五港镇', 80),
             ('保滩街道', 80),
             ('顺河镇', 91),
             ('范集镇', 92),
             

In [ ]:
df.head()

,BH,XSBH,WRLX,DSMC,XZQH,XSNR
0,004b0f4e1ca7a031db791fb17f30af77,201907030015,NaN,淮安市,淮阴区,淮阴区丁集镇镇北村8支桥下二大沟属区级管护沟，且从上游口流下大量的生活垃圾及水花生腾堵在桥下...
1,005042ac1dc448d43b4189b0bc6ca250,201908090037,大气污染,淮安市,新港街道,青年西路新港义务制学校工地内工房前裸土未覆盖
2,0050a0aeb4e0cdcbf462440c731e3721,202011240020,水污染,淮安市,市工业园区,淮安市朗坤污水处理有限公司（淮安市第三污水处理有限公司）\tPH仪器已安装联网。存在问题：废...
3,0050ee5e1847655e1a0865bb291b98ea,201911210010,大气污染,淮安市,生态文旅区,万康路与星辰路交叉口东南100m黄土露天堆放未覆盖，生活垃圾露天堆放未覆盖。
4,005494bc2c1bd85cab5da437bbebee50,202005020091,固废污染,淮安市,淮阴区,经巡查，发现当事人周雷驾驶渣土车在淮阴区回春路沿途泄露、抛洒。


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

df_sample = df.copy(deep=True)
# 投诉问题
df_sample['problem'] = df_sample['XSNR'].progress_apply(extract_problem)

  0%|          | 0/71849 [00:00<?, ?it/s]

In [ ]:
# 楼盘
df_sample['loupan'], df_sample['loupan_address'] = zip(*df_sample['problem'].progress_apply(extract_loupan))

  0%|          | 0/71849 [00:00<?, ?it/s]

In [ ]:
# 地址
df_sample['location'] = df_sample['problem'].progress_apply(extract_location)

  0%|          | 0/71849 [00:00<?, ?it/s]

In [ ]:
df_sample.to_excel(f'{unlabeled_data_path}/clean_data.xlsx', index=False)

In [ ]:
# 包含相似或相同命名实体（LOCATION/ORGANIZATION)或楼盘名称
from openpyxl import load_workbook

for i in list(xzqh_cnt.items())[1:]:
  sentence_pairs = []
  df_i = df_sample[df_sample['XZQH']==i[0]]
  for idx1, row1 in tqdm(df_i.iterrows(), total=len(df_i), desc=i[0]):
      for idx2, row2 in df_i.iterrows():
          result = find_similar_pairs(row1, row2)
          if idx1!=idx2 and result is not None: 
              sentence_pairs.append([row1['problem'], row2['problem']])
  
  if sentence_pairs:
    with pd.ExcelWriter(f'{unlabeled_data_path}/ner.xlsx', mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
        df_s = pd.DataFrame(sentence_pairs)
        df_s.to_excel(writer, sheet_name=i[0]) 

宝应湖农场:   0%|          | 0/5 [00:00<?, ?it/s]

清江浦区淮安现代商务集聚区管委会:   0%|          | 0/18 [00:00<?, ?it/s]

金港路街道:   0%|          | 0/20 [00:00<?, ?it/s]

清江浦区清河经济开发区管委会:   0%|          | 0/23 [00:00<?, ?it/s]

金湖经济开发区:   0%|          | 0/29 [00:00<?, ?it/s]

...

经济技术开发区:   0%|          | 0/4277 [00:00<?, ?it/s]

淮安区:   0%|          | 0/5893 [00:00<?, ?it/s]

淮阴区:   0%|          | 0/6841 [00:00<?, ?it/s]

盱眙县:   0%|          | 0/7067 [00:00<?, ?it/s]

清江浦区:   0%|          | 0/8711 [00:00<?, ?it/s]

In [ ]:
# sentence transformers余弦相似度相似
from sentence_transformers import SentenceTransformer, util
model_noft = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

for i in xzqh_cnt:
    print(i)
    sentences = df_sample[df_sample['XZQH']==i]['problem'].tolist()
    paraphrases = util.paraphrase_mining(model_noft, sentences, show_progress_bar=True, max_pairs=5000, top_k=50)

    sentence_pairs = []
    for paraphrase in paraphrases:
        score, i, j = paraphrase
        if score < .99:
          sentence_pairs.append([score, sentences[i], sentences[j]])

    #with pd.ExcelWriter(f'{unlabeled_data_path}/sbert.xlsx', mode="w", engine="openpyxl") as writer:
    with pd.ExcelWriter(f'{unlabeled_data_path}/sbert.xlsx', mode="a", engine="openpyxl", if_sheet_exists='new') as writer:
        df_s = pd.DataFrame(sentence_pairs)
        df_s.to_excel(writer, sheet_name=i) 